In [9]:
import pandas as pd
import numpy as np
import os
from tensorflow.keras.models import load_model
import joblib


In [21]:
loaded_model = load_model('trained_model.h5')
loaded_scaler = joblib.load('scaler.pkl')
df = pd.read_csv('./rawData/data_prueba_Forecasting_validation_data.csv')
df=df.drop(['Unnamed: 0'], axis=1)

def select_features(data):
    selected_features = ['Evento', 'Tienda', 'Producto', 'Mes', 'Day', 'Year', 'Formato_Hiper', 'Formato_Super',
                         'Subformato_Basico', 'Subformato_Intermedio', 'Subformato_Plus', 'DayOfWeek']
    return data[selected_features]

def create_sequences(data, sequence_length):
    sequences = []

    for i in range(len(data) - sequence_length + 1):
        sequence = data[i:i + sequence_length]
        sequences.append(sequence)
    return sequences

In [20]:
df.head()

,Venta,Evento,Tienda,Producto,id_fec_diaria,Precio,Mes,Day,Year,Formato_Hiper,Formato_Super,Subformato_Basico,Subformato_Intermedio,Subformato_Plus,DayOfWeek
0,1.5,75,1,369772,2021-10-15,26.030001,10,15,2021,0,1,1,0,0,4
1,1.0,76,1,369772,2021-10-16,26.030001,10,16,2021,0,1,1,0,0,5
2,4.0,77,1,369772,2021-10-17,26.030001,10,17,2021,0,1,1,0,0,6
3,3.0,78,1,369772,2021-10-18,25.636667,10,18,2021,0,1,1,0,0,0
4,2.5,79,1,369772,2021-10-19,25.693334,10,19,2021,0,1,1,0,0,1


In [22]:
X_new_selected = select_features(df)
X_new_scaled = loaded_scaler.transform(X_new_selected)

# Create sequences
sequence_length = 7
X_new_sequences = create_sequences(X_new_scaled, sequence_length)

# Predict using the model
predictions_scaled = loaded_model.predict(X_new_sequences)
predictions_original = loaded_scaler.inverse_transform(predictions_scaled)

# Evaluate the predictions
actual_venta = df['Venta'].values[sequence_length:]  # Extract actual Venta values
mae = mean_absolute_error(actual_venta, predictions_original)
print("Mean Absolute Error:", mae)

c:\Users\alan_\miniconda3\envs\scikitTensor\lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
